In [11]:
from oggm import utils
import geopandas as gpd
from oggm import cfg
from oggm import tasks, utils, workflow, graphics, DEFAULT_BASE_URL, global_task
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from oggm.global_tasks import compile_ela


In [12]:
def initializa_gdir(glacier_name = 'RGI60-15.03473') :
    # Initialize OGGM and set up the default run parameters
    cfg.initialize(logging_level='WARNING')

    # Local working directory (where OGGM will write its output)
    # WORKING_DIR = utils.gettempdir('OGGM_distr4')
    cfg.PATHS['working_dir'] = utils.get_temp_dir('OGGM_distributed', reset=True)

    rgi_ids = [glacier_name]  # This is Aletsch
    gdir = workflow.init_glacier_directories(rgi_ids, prepro_base_url=DEFAULT_BASE_URL, from_prepro_level=4, prepro_border=80)[0]
    return gdir

In [18]:
def compute_ela(glacier_name='RGI60-15.03473', ys=1901, ye=2019):
    gdir = initializa_gdir(glacier_name=glacier_name)
    donnees = compile_ela(gdir, ys=ys, ye=ye)
    return donnees

In [14]:
def compute_mean_by_period(donnees, glacier_name, period = "10Y") :
    donnees = pd.DataFrame({
        'annee': [i for i in range(1901, 2020)],
        'donnees': donnees[glacier_name]
    })

    donnees['annee'] = pd.to_datetime(donnees['annee'], format='%Y')

    donnees = donnees.set_index('annee')

    moyenne_ans = donnees['donnees'].resample(period).mean()

    return moyenne_ans

In [ ]:
#Calcul des ela nécessaires
mdg_ela=compute_mean_by_period(compute_ela(glacier_name='RGI60-11.03643'),'RGI60-11.03643',"5Y")
lasneous_ela=compute_mean_by_period(compute_ela(glacier_name='RGI60-11.03234'),'RGI60-11.03234',"5Y")
ross_ela=compute_mean_by_period(compute_ela(glacier_name='RGI60-19.02266'),"RGI60-19.02266","5Y")
perito_ela=compute_mean_by_period(compute_ela(glacier_name='RGI60-17.00312'),"RGI60-17.00312","5Y")

In [ ]:
plt.plot(mdg_ela)
plt.plot(lasneous_ela)
plt.plot(ross_ela)
plt.plot(perito_ela)

In [ ]:
#Lissage des moyennes de l'ELA sur 5 ans
degree=40

x=np.arange(1900,2021,5)
coefficients=np.polyfit(x,perito_ela,degree)
coefficients=np.squeeze(coefficients)

polynomial=np.poly1d(coefficients)

fitted_curve=polynomial(x)

In [ ]:
#Affichage
plt.plot(x,fitted_curve)
plt.xlabel("Années")
plt.ylabel("Altitude(m)")
plt.title("Ligne d'équilibre de Perito Moreno en fonction des années")